<a href="https://colab.research.google.com/github/duskvirkus/ml-art-colab/blob/main/ffhq_align_faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Align Faces
This notebook will walk thru the process to take a folder of face images and convert them to images in the format of FFHQ.

In [ ]:
!git clone https://github.com/dvschultz/stylegan2-ada

In [ ]:
!gdown --id 1iW2ts9PAWa_5_B3LS16KvpvAvoeLZc8v
!unzip /content/suicidegirls-jpg.zip

In [ ]:
%cd /content/stylegan2-ada/

In [ ]:
import os
import sys
import bz2
from keras.utils import get_file
from ffhq_dataset.face_alignment import image_align
from ffhq_dataset.landmarks_detector import LandmarksDetector

LANDMARKS_MODEL_URL = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'
RAW_IMAGES_DIR = '/content/suicidegirls-jpg'
ALIGNED_IMAGES_DIR = '/content/drive/MyDrive/datasets/suicidegirls-faces'

if not os.path.exists(ALIGNED_IMAGES_DIR):
        os.makedirs(ALIGNED_IMAGES_DIR)


def unpack_bz2(src_path):
    data = bz2.BZ2File(src_path).read()
    dst_path = src_path[:-4]
    with open(dst_path, 'wb') as fp:
        fp.write(data)
    return dst_path

landmarks_model_path = unpack_bz2(get_file('shape_predictor_68_face_landmarks.dat.bz2', LANDMARKS_MODEL_URL, cache_subdir='temp'))


landmarks_detector = LandmarksDetector(landmarks_model_path)
for img_name in [x for x in os.listdir(RAW_IMAGES_DIR) if x[0] not in '._']:
    raw_img_path = os.path.join(RAW_IMAGES_DIR, img_name)
    for i, face_landmarks in enumerate(landmarks_detector.get_landmarks(raw_img_path), start=1):
        face_img_name = '%s_%02d.png' % (os.path.splitext(img_name)[0], i)
        aligned_face_path = os.path.join(ALIGNED_IMAGES_DIR, face_img_name)
        os.makedirs(ALIGNED_IMAGES_DIR, exist_ok=True)
        image_align(raw_img_path, aligned_face_path, face_landmarks)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/datasets/
!zip suicidegirls-faces.zip ./suicidegirls-faces -r

In [ ]:
!cp ./suicidegirls-faces.zip /content/drive/MyDrive/datasets/